Гафни Даниил, 306 группа, физический факультет МГУ

### Импортируем необходимые библиотеки

In [2]:
import numpy as np

import plotly
import plotly.graph_objs as go

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation

import warnings

from IPython.display import clear_output, HTML, display

### Проведем необходимые настройки

In [3]:
plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [156]:
# Метод прогонки
def tridiagonal_matrix_algorithm(coeffs, F):
    '''
    Метод прогонки.
    
    Параметры: 
        coeffs (numpy.array): Тригианональная матрица коэффициентов уравнений.
        F (numpy.array): Массив правых частей уравнений.
    '''
    N = coeffs.shape[0]-1
    Y = np.zeros(N)
    coeffs = coeffs[1:-1]
    
    kappa1, nu1 = coeffs[0][0], coeffs[0][1]
    kappa2, nu2 = coeffs[-1][-2], coeffs[-1][-1]
    alpha = np.zeros(N)
    beta = np.zeros(N)
    alpha[0] = kappa1
    beta[0] = nu1
    A = np.zeros(N-1)
    B = np.zeros(N-1)
    C = np.zeros(N-1)
    for i, line in enumerate(coeffs):
        A[i] = line[i]
        B[i] = line[i+1]
        C[i] = line[i+2]
    
    for n in range(1, len(alpha)):
        alpha[n] = -B[n-1]/(C[n-1] + alpha[n-1]*A[n-1])
        beta[n] = -(F[n-1] + A[n-1]*beta[n-1])/(C[n-1] + alpha[n-1]*A[n-1])
        
    Y[N-1] = (nu2 + kappa2*beta[N-1])/(1 - alpha[N-1]*kappa2)
    for n in range(len(Y)-2, -1, -1):
        Y[n] = alpha[n+1]*Y[n+1] + beta[n+1]
    return Y

In [157]:
coeffs = np.array([[1, 2, 0, 0, 0, 0], [4, 5, 6, 0, 0, 0], [0, 7, 8, 9, 0, 0],
                   [0, 0, 10, 11, 12, 0], [0, 0, 0, 13, 14, 15], [0, 0, 0, 0, 16, 17]])
F = [5, 20, 30, 40]

tridiagonal_matrix_algorithm(coeffs, F)

array([-1.09852053, -0.16650967, -1.35147116, -1.10156813, -0.42195379])